# Data Integration Project
## Movie Box Office Analysis


#### Project Members: Hafsa Redouane, Wahiba Sbai, Najlaa Allioui, Sarah Shahin

## Summary

1) Objective and context of the DW and NoSQL integration.
2) Schema design and implementation.
3) Description and motivation for the queries.
4) Analysis of results and visualization.
5) Clustering analysis and conclusions.


## 1. Objective and context of the DW and NoSQL integration.



### a.Context of the Dataset
The **Movies Box Office Dataset (2000-2024)** provides a comprehensive overview of **global box office performance** over the past two decades. It includes key financial and categorical data related to **movie revenues, production costs, genres, audience ratings, and linguistic distributions**. The dataset enables the analysis of **financial profitability, audience preferences, and market trends in the film industry**.  


<img src="presentation.jpg" width="400">



### Key Features of the Dataset:
- **Timeframe:** Covers movies released from **2000 to 2024**.  
- **Metrics:** Includes **worldwide revenue, domestic and foreign gross, profit margins**.  
- **Genres:** Provides classification into **multiple genres per movie**, enabling trend analysis.  
- **Audience Ratings:** Includes **average ratings and vote counts** from various platforms.  
- **Language and Country Information:** Specifies the **original language** and **production countries** of each movie.  


## b.Objective of the Data Warehouse (DW)
The objective of building a Data Warehouse (DW) for this dataset is to structure and optimize data storage for efficient querying and analysis. Instead of dealing with unstructured movie data, a dimensional model (snowflake schema) is used to allow fast OLAP queries, data aggregation, and trend analysis.  

The Data Warehouse is designed to optimize data analysis by structuring information for trend exploration, forecasting, and visualization. It enables aggregated metric calculations, such as total revenue per genre, highest-grossing movies by country, and profitability trends over the years. With support for OLAP queries—including CUBE, ROLLUP, and GROUP BY—the system facilitates multi-dimensional data exploration, allowing for in-depth insights into revenue distribution and market performance.


## c.NoSQL Integration with Neo4j
While the Data Warehouse (DW) handles structured data, the relationships between movies, genres, and production companies can be better represented using a graph database (NoSQL). This is where Neo4j comes in. 

Neo4j is highly optimized for handling complex relationships, making it ideal for managing many-to-many connections between movies, genres, and production countries, which can be challenging in traditional relational databases. Its efficient graph queries allow for fast traversal of relationships, enabling advanced searches such as "Find all action movies produced in Canada that generated over $500M in revenue" with greater speed and flexibility. Additionally, its graph-based structure provides a real-world representation of the film industry by accurately modeling the connections between movies, actors, genres, and studios, making data exploration more intuitive and insightful.


## 2. Schema design and implementation.


### a.Data WareHouse(MySql)

### Main Tables:  
- **Fact_Movie**: Contains essential information about movies, including title, release year, revenue, rating, and other key attributes.  
- **Dim_Genre**: Stores movie genres, enabling clear classification and analysis.  
- **Movie_Genre**: Manages the many-to-many relationship between movies and their associated genres.  
- **Dim_ProductionCountry**: Lists the production countries for movies.  
- **Movie_ProductionCountry**: Establishes the links between movies and their respective production countries.

<img src="shema.png" width="700">

### **SQL Code:**  

The following script creates the database and the necessary tables for its proper functioning.  


In [ ]:
CREATE DATABASE IF NOT EXISTS movies;
USE movies;

-- Movies Table (Fact Table)
CREATE TABLE Fact_Movie (
    movie_id INT AUTO_INCREMENT PRIMARY KEY,
    title VARCHAR(255),
    release_year INT,
    worldwide_revenue BIGINT,
    domestic_revenue BIGINT,
    domestic_percentage DECIMAL(5,2),
    foreign_revenue BIGINT,
    foreign_percentage DECIMAL(5,2),
    rating DECIMAL(3,1),
    vote_count INT,
    original_language VARCHAR(10)
);

-- Genre Table (Dimension)
CREATE TABLE Dim_Genre (
    genre_id INT AUTO_INCREMENT PRIMARY KEY,
    genre_name VARCHAR(100) UNIQUE
);

-- Many-to-Many Relationship Table between Movies and Genres
CREATE TABLE Movie_Genre (
    movie_id INT,
    genre_id INT,
    PRIMARY KEY (movie_id, genre_id),
    FOREIGN KEY (movie_id) REFERENCES Fact_Movie(movie_id) ON DELETE CASCADE,
    FOREIGN KEY (genre_id) REFERENCES Dim_Genre(genre_id) ON DELETE CASCADE
);

-- Production Countries Table (Dimension)
CREATE TABLE Dim_ProductionCountry (
    country_id INT AUTO_INCREMENT PRIMARY KEY,
    country_name VARCHAR(100) UNIQUE
);

-- Many-to-Many Relationship Table between Movies and Production Countries
CREATE TABLE Movie_ProductionCountry (
    movie_id INT,
    country_id INT,
    PRIMARY KEY (movie_id, country_id),
    FOREIGN KEY (movie_id) REFERENCES Fact_Movie(movie_id) ON DELETE CASCADE,
    FOREIGN KEY (country_id) REFERENCES Dim_ProductionCountry(country_id) ON DELETE CASCADE
);

### b.Neo4j

<img src="neo.png" width="700">

This schema represents how movies are connected to different entities in our dataset.

- **BELONGS_TO (Movie → Genre): Links movies to their genres (e.g., Inception → Sci-Fi).
- **PRODUCED_IN (Movie → Country): Shows where a movie was produced (e.g., Parasite → South Korea).
- **LANGUAGE_IS (Movie → Language): Indicates the movie’s primary language (e.g., La La Land → English).
- **HAS_REVIEW (Movie → Review): Connects movies to user reviews and ratings.

### **2. Installing Dependencies**  

To ensure communication between Python and MySQL, we installed **pymysql**, which allows executing SQL queries from a Python script.  

```bash

In [ ]:
pip install pymysql


### **3. Data Loading and Cleaning**  

We used **pandas** to load a CSV file containing detailed information about movies. This step ensures that only clean and usable data is inserted into the database.  

**Operations performed:**  
- Removal of missing values to prevent errors during insertion.  
- Renaming columns to ensure consistency with the database schema.  
- Converting text fields into appropriate formats (e.g., separating genres and production countries for better data normalization).

### **Python Code:**  

This script loads data from the CSV file and inserts it into the MySQL database.

In [ ]:
import pymysql
import pandas as pd

# Chargement des données
print("Chargement des données...")
df = pd.read_csv("movies_box_office.csv").dropna()
print("Données chargées avec succès.")

# Connexion à MySQL
print("Connexion à la base de données...")
conn = pymysql.connect(host="localhost", user="root", password="", database="movies")
cursor = conn.cursor()
print("Connexion réussie.")

# Insertion des données
print("Insertion des données en cours...")
for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO Fact_Movie (title, release_year, worldwide_revenue, domestic_revenue, 
                                domestic_percentage, foreign_revenue, foreign_percentage, 
                                rating, vote_count, original_language)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (row["title"], row["release_year"], row["worldwide_revenue"], row["domestic_revenue"],
           row["domestic_percentage"], row["foreign_revenue"], row["foreign_percentage"],
           row["rating"], row["vote_count"], row["original_language"]))
print("Insertion terminée.")

# Validation et fermeture de la connexion
conn.commit()
cursor.close()
conn.close()
print("Connexion fermée.")


## 3. Description and motivation for the queries 

***We designed and executed various queries to facilitate data insertion, relationship modeling, and analytical insights across both relational and graph databases.***  

- **Data Insertion Queries (MySQL):**  
  - We processed the **movies_box_office.csv** file and inserted movie details into the **Fact_Movie** table.  
  - **Genre and production country relationships** were managed using **Dim_Genre** and **Dim_ProductionCountry**, ensuring proper normalization and avoiding data redundancy.  
  - We utilized **`INSERT IGNORE` statements** to prevent duplicate entries and maintain consistency across linked tables.  

- **Graph Database Queries (Neo4j - Cypher):**  
  - We imported movies into Neo4j using **`LOAD CSV WITH HEADERS`**, enabling structured ingestion of unstructured data.  
  - **Relationships between movies, genres, and production countries** were established using **Cypher queries**, allowing efficient traversal and retrieval.  
  - We stored additional metadata for unstructured analysis, highlighting the flexibility of graph databases in handling diverse data formats.  

- **Analytical and Aggregation Queries:**  
  - We executed **OLAP-style queries (CUBE, ROLLUP)** to aggregate revenue based on different dimensions, such as year, genre, and country.  
  - **Correlation analysis queries** helped us explore relationships between movie ratings, revenue, and production origins.  
  - We performed **clustering queries** using k-means segmentation to group movies based on revenue and rating patterns.  

These queries collectively enhance data exploration and support meaningful insights, demonstrating the advantages of integrating **MySQL for structured storage** and **Neo4j for relationship-driven analysis**. 




###  4. Analysis of Results and Visualization 

We applied various visualization techniques to better analyze trends, relationships between entities, and key insights from the dataset.  

**Techniques used:**  
- **OLAP Analysis and Aggregations:** Using **CUBE** and **ROLLUP** queries, data matrices were generated to explore revenue aggregates (global, domestic, and foreign) based on movie release years and genre classifications.  

- **Graphical Representation with Python:** Different visualizations were created using **Matplotlib** and **Seaborn** to highlight key insights:  
  - **Histograms** to illustrate the distribution of movie revenues.   
  - **Scatter plots** to analyze the correlation between movie ratings and worldwide revenue.  


Example: 

<img src="avenger.png" >


- **Graph Database Visualization with Neo4j:** Interactive graph visualizations were created using **Cypher** queries to showcase relationships between movies, genres, and production countries. This approach facilitates understanding complex connections, such as industry influences on global box office performance. 

example:

<img src="bloom.png" >



This graph shows the **Neo4j Bloom interface**, which provides a powerful way to explore the graph database visually.
We can see here all the movies that belong to the **Romance Categorie**.

 

- **Clustering Analysis:** A **k-means clustering** analysis was conducted to segment movies based on revenue and ratings, helping to distinguish different groups based on commercial and critical success.  


- **Graph Database Visualization with Neo4j:**  
  - We created interactive graph visualizations to explore relationships between movies, genres, and production countries.  
  - This allowed us to analyze network-based trends, such as the influence of certain film industries on global box office performance.  

- **Clustering Analysis:**  
  - Using **k-means clustering**, we segmented movies into different groups based on revenue and ratings.  
  - This helped us classify movies into high-grossing blockbusters, critically acclaimed films, and lower-performing releases.  


### 5. Clustering Analysis and Conclusions

***We performed clustering analysis to group movies based on revenue and ratings, uncovering patterns that define different categories of films.***  

- **K-Means Clustering for Movie Segmentation:**  
  - We applied **k-means clustering** to divide movies into groups based on **worldwide revenue and ratings**.  
  - This helped us identify trends among different types of movies, such as:  
    - **Blockbusters** (high revenue, high ratings).  
    - **Critically acclaimed films** (low revenue, high ratings).  
    - **Commercial failures** (low revenue, low ratings).  

- **Feature Selection for Clustering:**  
  - We used **rating, vote count, and revenue** as clustering variables.  
  - Preprocessing steps included standardizing data to ensure fair distance calculations.  

- **Cluster Visualization and Interpretation:**  
  - The clusters were visualized using **scatter plots**, highlighting the separation between different types of movies.  
  - The results demonstrated that high-budget productions tend to have higher revenue, but ratings do not always correlate with financial success.  

<img src="vis.png" >

| *Cluster #* | *Corresponds to* | *Key Characteristics* |
|--------------|------------------|-------------------|
| *Cluster 1* | *High-revenue blockbusters* | Movies with *high revenue, **high vote count, genres **Action, Sci-Fi, Adventure, mainly produced in **US/UK* |
| *Cluster 2* | *Moderate-budget well-received movies*  | Medium revenue, genres *Comedy, Drama, Romance, produced in a mix of **Europe/Asia* |
| *Cluster 3* | *Niche movies or horror/thriller* | Low revenue, genres *Horror, Thriller, Independent Drama*, targeting a smaller audience |
| *Cluster 4* | *Very unpopular movies* | Very low revenue, few votes, genres *Documentary, Experimental*, often from emerging countries |



**Cluster 1**: Films with high worldwide and domestic revenue, a high number of votes, and a high rating.

- **Characteristics**: These films are generally popular blockbusters, with wide international distribution.
- **Genres**: Mainly popular genres like Action, Science Fiction, Adventure, etc.
- **Production Countries**: Mostly films produced in the United States and the United Kingdom.


**Cluster 2**: Films with medium revenue, moderate ratings, and fewer votes.

- **Characteristics**: These films are likely to belong to the medium-budget category, but still well-received by the audience.
- **Genres**: Genres like Comedy, Drama, and Romance, which attract a more specific audience.
- **Production Countries**: A mix of films produced in European or Asian countries.


**Cluster 3**: Films with low revenue and fewer votes, but with niche genres.

- **Characteristics**: These films may be less popular globally but may have a niche audience or a loyal fanbase.
- **Genres**: Genres like Horror, Thriller, or Independent Drama, which attract a smaller, more specific audience.
- **Production Countries**: Emerging countries or low-budget productions are likely predominant here.



**Cluster 4**: Films with low revenue, lower ratings, and very few votes.

- **Characteristics**: These films are very unpopular, likely low-budget, and have limited international distribution.
- **Genres**: Often experimental genres, Documentaries, or films that are highly specific to certain cultures.
- **Production Countries**: Films produced in emerging countries or with very limited budgets.


